## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [ ]:
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr

c:\Users\lukag\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

import google.generativeai as genai
from dotenv import load_dotenv





In [4]:
load_dotenv()
api_key = os.getenv("gemini_key")
genai.configure(api_key=api_key)


In [5]:
from langchain_community.document_loaders import PyMuPDFLoader
import glob
import os

# Ruta al directorio
folder = glob.glob("C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/Diabetes_docs/")[0]

# Cargar todos los PDFs en ese directorio
documents = []
for pdf_path in glob.glob(os.path.join(folder, "*.pdf")):
    print(f"Cargando documento: {pdf_path}")
    loader = PyMuPDFLoader(pdf_path)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = "PDF"
        documents.append(doc)

print(f"Se han cargado {len(documents)} documentos correctamente.")


Cargando documento: C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/Diabetes_docs\20-7-1183.pdf
Cargando documento: C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/Diabetes_docs\21-9-1414.pdf
Cargando documento: C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/Diabetes_docs\654.pdf
Cargando documento: C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/Diabetes_docs\NEJM199309303291401.pdf
Cargando documento: C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/Diabetes_docs\NEJM199311113292004.pdf
Cargando documento: C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Di

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [7]:
len(chunks)

108

In [8]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: PDF


## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [9]:
import onnx, onnxruntime

print("ONNX:", onnx.__version__)
print("ONNX Runtime:", onnxruntime.get_device())  # debería mostrar CPU o GPU


ONNX: 1.18.0
ONNX Runtime: GPU


In [10]:
import onnx 
import onnxruntime
print(onnxruntime.__version__)


1.22.0


In [13]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite

db_name = "C:/Users/lukag/OneDrive/Desktop/Universidad/3ero/cuadrimestre2/PAID/github/IDSS-for-Diabetes-Readmission-Prediction/src/agent_2/db_place"

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Delete if already exists

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 108 documents


In [14]:
# Get one vector and find how many dimensions it has

collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 384 dimensions


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [17]:
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]

colors = ["blue"] * len(documents)


In [ ]:
# # Prework

# result = collection.get(include=['embeddings', 'documents', 'metadatas'])
# vectors = np.array(result['embeddings'])
# documents = result['documents']
# doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
# colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [18]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [19]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [23]:
%pip install --upgrade langchain-google-genai pillow


  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached langchain_google_genai-2.1.4-py3-none-any.whl (44 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 2.7/2.7 MB 38.3 MB/s eta 0:00:00

   ---------------------------------------- 0/4 [filetype]
  Attempting uninstall: pillow
   ---------------------------------------- 0/4 [filetype]
    Found existing installation: pillow 11.1.0
   ---------------------------------------- 0/4 [filetype]
    Uninstalling pillow-11.1.0:
   ---------------------------------------- 0/4 [filetype]
      Successfully uninstalled pillow-11.1.0
   --------------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [25]:
os.environ["GOOGLE_API_KEY"] = api_key

In [28]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA

# 1. Configura el retriever (igual que antes)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2. Instancia Gemini Pro (texto)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# 3. Construye la cadena RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",      # o 'map_reduce', 'refine', según necesites
    retriever=retriever
)




In [33]:
# 4. Función de consulta
def answer_with_rag(prompt: str) -> str:
    return qa_chain.run(prompt)

# Ejemplo
respuesta = answer_with_rag("¿de que son los documentos proporcionados?")
print(respuesta)

Los documentos proporcionados son sobre diabetes. Uno de ellos es un apéndice con datos estadísticos sobre la prevalencia de diabetes en varios países de América, incluyendo datos de población, prevalencia de diabetes en porcentaje, y número de personas afectadas por área (rural, urbana) y por género y edad. El otro documento es un artículo sobre la definición, clasificación, diagnóstico, screening y prevención de la diabetes mellitus.


# Time to use LangChain to bring it all together

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">PLEASE READ ME! Ignoring the Deprecation Warning</h2>
            <span style="color:#900;">When you run the next cell, you will get a LangChainDeprecationWarning 
            about the simple way we use LangChain memory. They ask us to migrate to their new approach for memory. 
            I feel quite conflicted about this. The new approach involves moving to LangGraph and getting deep into their ecosystem.
            There's a fair amount of learning and coding in LangGraph, frankly without much benefit in our case.<br/><br/>
            I'm going to think about whether/how to incorporate it in the course, but for now please ignore the Depreciation Warning and
            use the code as is; LangChain are not expected to remove ConversationBufferMemory any time soon.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# create a new Chat with OpenAI
llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
query = "Can you describe Insurellm in a few sentences"
result = conversation_chain.invoke({"question":query})
print(result["answer"])

In [ ]:
# set up a new conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# putting it together: set up the conversation chain with the GPT 4o-mini LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

## Now we will bring this up in Gradio using the Chat interface -

A quick and easy way to prototype a chat with an LLM

In [ ]:
# Wrapping in a function - note that history isn't used, as the memory is in the conversation_chain

def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
# And in Gradio:

view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)